In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Orchestrating Multi-Agent Systems with Google Agent Development Kit

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi-agent_systems.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fiamthuya%2Fgoogle-cloud-workshops%2Fmain%2Fai-agents%2Fagent-development-kit%2Forchestrating_multi-agent_systems.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi-agent_systems.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/iamthuya/google-cloud-workshops/blob/main/ai-agents/agent-development-kit/orchestrating_multi-agent_systems.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Thu Ya Kyaw](https://github.com/iamthuya) |

## Environment Setup

### Install Necessary Libraries

Here you will install required Python packages for this lab.

In [ ]:
!pip install -U -q google-adk

In [ ]:
# Check ADK version
from google import adk

print(adk.__version__)

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

Don't worry if you see a notification message like `Your session crashed for an unknown reason.` This is expected as you are shutting down the kernel from the same instance using code.


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Import Required Libraries

In [1]:
import os
import asyncio

from google.adk.agents import LlmAgent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, ToolContext
from google.genai import types

from IPython.display import Markdown, display

### Define Environment Variables

In [2]:
from google.colab import userdata

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

## Sequential Agents

In this section, you'll create an agent that researches a topic, then writes in detail about it, and finally edits the output. These steps must happen in order: research first, then writing, and then editing, because you can't write without research, and you can't edit without a written piece.

In [3]:
# Define variables
GEMINI_MODEL_NAME = "gemini-2.5-flash-preview-04-17"
APP_NAME = "sequential_app"
USER_ID = "sequential_user"
SESSION_ID = "sequential_session"

#### Initialize the agents and pipeline

In [4]:
# Define respective agents and pipeline
researcher = LlmAgent(
    name="ResearcherAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to use 'google_search' tool to find all the relavent information about a given topic and write a comprehensive report about that topic.",
    tools=[google_search],
    description="Researches about a topic",
    output_key="research_results"
)

writer = LlmAgent(
    name="WriterAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to write a comprehensive report using this information: {research_results}. Make sure to expend on all the topics from the provided information.",
    description="Write about a comprehensive report based on the provided information",
    output_key="comprehensive_report"
)

editor = LlmAgent(
    name="EditorAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to edit a report: {comprehensive_report}. Perform a quality check on the text and improve it if necessary.",
    description="Perform quality check on a written report",
)

sequential_pipeline = SequentialAgent(
    name="ContentPipelineAgent",
    sub_agents=[researcher, writer, editor],
    description="Executes a sequence of researching, writing, and editing.",
)

### Initialize Session & Runner

In [5]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Initialize Runner
sequential_runner = Runner(
    agent=sequential_pipeline,
    app_name=APP_NAME,
    session_service=session_service,
)

### Define a function to interact with Agent

In [20]:
# Define a function to interact with the agent
async def run_agent(runner, user_id, session_id, user_query):
    user_message = types.Content(
        role="user",
        parts=[types.Part(text=user_query)]
    )

    events = runner.run_async(
        user_id=user_id,
        session_id=session_id,
        new_message=user_message
    )

    output = {}
    async for event in events:
        if event.is_final_response():
            author = event.author
            report = event.content.parts[0].text
            output[author] = report

            display(Markdown(f"## {author}"))
            display(Markdown(f"{report}"))

    return output

In [9]:
topic = "Write about the current state of generative ai"
output = await run_agent(sequential_runner, USER_ID, SESSION_ID, topic)
report = output['EditorAgent']

## ResearcherAgent

## The Current State of Generative AI: A Transformative Technology in Rapid Evolution (as of early 2025)

Generative Artificial Intelligence (AI), capable of creating novel content such as text, images, audio, and code, has rapidly evolved from a burgeoning technology to a significant transformative force across numerous industries. Unlike traditional AI that primarily focuses on analyzing or classifying existing data, generative AI learns patterns from large datasets to produce entirely new outputs. This paradigm shift is being driven by advanced models like Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and particularly Large Language Models (LLMs).

The current state of generative AI is marked by widespread and accelerating adoption in both business and consumer spheres. Organizations are moving beyond initial experimentation towards more meaningful integration of generative AI into their core workflows and enterprise applications. This is reflected in significant increases in investment in generative AI technologies by businesses. Generative AI features are increasingly being embedded into existing software and services, appearing as conversational interfaces, digital assistants, chatbots, and copilots.

A key trend defining the current landscape is the rise of multimodal AI models, which can seamlessly process and generate content across different modalities, including text, images, audio, and even 3D content. This capability is expanding the potential applications of generative AI significantly. Furthermore, generative AI is enabling unprecedented levels of hyper-personalization in diverse sectors such as marketing, e-commerce, healthcare, and gaming, tailoring experiences to individual users. Real-time generative applications are also becoming more prevalent, facilitating tasks like instant language translation, dynamic content creation for live events, and responsive customer service interactions.

Looking ahead, the emergence of autonomous AI agents powered by generative models is anticipated to become more commonplace. These agents are designed to handle complex tasks and make decisions with minimal human intervention. There is also a growing focus on applying generative AI to solve domain-specific problems within particular industries, leading to an acceleration in the development of vertical AI applications. Advancements in hardware and increased computational power are also contributing to the rapid progress and expanding capabilities of generative AI.

Despite its rapid advancements and transformative potential, generative AI faces several significant limitations and challenges. A major concern is the technology's heavy reliance on training data, which can perpetuate and even amplify existing biases present in the datasets, potentially leading to discriminatory or irrelevant outputs, particularly for underrepresented groups or regions. The issue of "hallucinations," where models generate plausible-sounding but inaccurate or fabricated information, remains a critical challenge, impacting the reliability and trustworthiness of AI-generated content. Unlike humans, generative AI lacks an inherent "source of truth" and struggles with nuanced contextual understanding and complex reasoning.

The substantial computational resources and energy required for training and running large generative models pose significant cost and environmental sustainability concerns. Ethical and privacy issues are also paramount, including the potential for data leakage, misuse of personal data, lack of transparency in decision-making processes, and the generation of harmful or misleading content like deepfakes and misinformation. Furthermore, challenges exist in seamlessly integrating generative AI with existing systems, and its adoption can be particularly difficult for small and medium-sized enterprises. Concerns surrounding intellectual property rights and the potential for AI-generated content to infringe on copyrighted material are also being navigated. Language barriers and performance disparities between different languages remain limitations for some models.

The increasing capabilities of generative AI also raise broader societal concerns, such as the potential erosion of human creativity through over-reliance on AI tools and the risk of job displacement in creative and knowledge-based industries. Moreover, the technology can amplify existing safety and security risks by providing threat actors with enhanced capabilities for scams, fraud, and disinformation campaigns.

In response to these challenges, there is a growing emphasis on developing ethical AI frameworks, mitigating bias in training data and models, increasing transparency in how AI systems generate outputs, and establishing robust governance structures. Efforts are also underway to improve the energy efficiency of AI models and infrastructure. Organizations are recognizing the need to invest in reskilling and upskilling their workforce to effectively collaborate with AI technologies.

In conclusion, generative AI is currently in a dynamic state of rapid advancement and increasing integration across various sectors. While offering immense potential for innovation, productivity gains, and personalized experiences, it also presents significant technical, ethical, and societal challenges that require careful consideration and proactive mitigation strategies for its responsible and beneficial development and deployment.

## WriterAgent

Okay, here is a comprehensive report on the current state of Generative AI based on the information provided.

---

**Comprehensive Report: The Current State of Generative AI (as of early 2025)**

**Executive Summary**

As of early 2025, Generative Artificial Intelligence (AI) stands as a rapidly evolving and profoundly transformative technology. Moving beyond its initial stages, it is now being widely adopted across numerous industries, integrating into core business processes and consumer applications. Driven by advanced models like LLMs and increasingly multimodal capabilities, generative AI offers significant potential for innovation, personalization, and productivity. However, its rapid advancement is accompanied by substantial challenges, including data biases, "hallucinations," high computational costs, ethical and privacy concerns, and broader societal impacts such as job displacement and security risks. Addressing these limitations through ethical frameworks, governance, and focused mitigation strategies is crucial for realizing the technology's potential responsibly.

**1. Introduction to Generative AI**

Generative Artificial Intelligence is a sophisticated form of AI distinguished by its capacity to create novel content, a stark contrast to traditional AI which primarily focuses on analyzing or classifying existing data. This new content can manifest in various forms, including text, images, audio, code, and even 3D structures. The fundamental mechanism involves learning patterns, structures, and distributions from vast datasets to produce outputs that are entirely new yet consistent with the training data's characteristics. This paradigm shift is largely propelled by significant advancements in underlying model architectures, particularly Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and most prominently, Large Language Models (LLMs).

**2. Current State of Adoption and Integration**

The current landscape of generative AI is characterized by widespread and rapidly accelerating adoption across both business and consumer domains. Organizations are transitioning beyond initial exploratory phases and are increasingly focused on the meaningful integration of generative AI into their core workflows and enterprise-level applications. This strategic shift is evidenced by substantial increases in investment dedicated to generative AI technologies by businesses across sectors. A key aspect of this adoption is the embedding of generative AI features directly into existing software and services. These features commonly appear as intuitive conversational interfaces, sophisticated digital assistants, helpful chatbots, and powerful "copilots" designed to augment human capabilities in various tasks.

**3. Key Trends and Capabilities Shaping the Landscape**

Several key trends are defining the contemporary generative AI landscape:

*   **Rise of Multimodal AI:** A significant development is the emergence of multimodal AI models. These advanced models possess the capability to seamlessly process and generate content across multiple distinct modalities simultaneously, such as integrating text and images, audio and video, or even incorporating 3D data. This cross-modal understanding and generation capability vastly expands the range of potential applications for generative AI.
*   **Hyper-Personalization:** Generative AI is enabling unprecedented levels of hyper-personalization across diverse industries. Sectors like marketing, e-commerce, healthcare, and gaming are leveraging generative models to tailor content, recommendations, and experiences precisely to the individual preferences and needs of users.
*   **Real-Time Applications:** The increasing efficiency and speed of generative models are leading to a greater prevalence of real-time generative applications. These applications facilitate dynamic tasks such as instant language translation during conversations, the rapid creation of dynamic content for live events, and providing responsive and context-aware interactions in customer service scenarios.

**4. Future Outlook and Emerging Areas**

Looking towards the near future, several developments are anticipated to become more prominent:

*   **Autonomous AI Agents:** The emergence and increasing commonplace nature of autonomous AI agents powered by generative models are expected. These agents are designed with the capability to handle complex tasks and make decisions autonomously, requiring minimal human intervention.
*   **Domain-Specific/Vertical AI:** There is a discernible trend towards focusing generative AI applications on solving specific problems within particular industries. This focus on "vertical AI" is accelerating the development of specialized generative models and applications tailored to the unique requirements and data of sectors like healthcare, finance, or manufacturing.
*   **Hardware and Computational Advancements:** Continued advancements in hardware, particularly in specialized AI processors and infrastructure, coupled with increasing computational power, are playing a crucial role. These advancements are contributing significantly to the rapid progress, enhanced capabilities, and scalability of generative AI models.

**5. Significant Limitations and Challenges**

Despite its rapid advancements and transformative potential, generative AI currently faces several significant limitations and challenges that impede its flawless implementation and widespread trustworthy deployment:

*   **Data Reliance and Bias:** A major challenge stems from the technology's heavy reliance on the data used for training. Existing biases present within these vast datasets can be perpetuated and even amplified by the models, potentially leading to discriminatory, unfair, or irrelevant outputs, particularly affecting underrepresented groups or regions.
*   **Hallucinations and Trustworthiness:** The phenomenon of "hallucinations," where models generate plausible-sounding but entirely inaccurate, fabricated, or nonsensical information, remains a critical issue. This directly impacts the reliability and trustworthiness of AI-generated content, requiring careful verification.
*   **Lack of Truth Source and Reasoning:** Unlike humans, generative AI models inherently lack an independent "source of truth." They struggle with genuinely nuanced contextual understanding and complex, multi-step reasoning, often relying on statistical patterns rather than causal logic or verified facts.
*   **Computational Resources and Sustainability:** Training and running large generative models demand substantial computational resources, leading to significant costs and raising concerns regarding environmental sustainability due to high energy consumption.
*   **Ethical and Privacy Issues:** Paramount ethical and privacy concerns include the potential for accidental or malicious data leakage, the misuse of personal data used in training or inputs, a lack of transparency in how models arrive at their outputs, and the potential for generating harmful or misleading content such as sophisticated deepfakes and widespread misinformation.
*   **Integration Difficulties:** Seamlessly integrating generative AI capabilities with existing legacy systems within organizations can be challenging, posing a particular barrier for small and medium-sized enterprises (SMEs) with limited technical resources.
*   **Intellectual Property Concerns:** Navigating intellectual property rights is complex, raising questions about the ownership of AI-generated content and the potential for generative models to infringe on copyrighted material used, sometimes inadvertently, in their training data or output generation.
*   **Language Barriers:** While progress is being made, performance disparities and limitations often persist between different languages, with models typically performing best in languages heavily represented in training data.

**6. Broader Societal Concerns**

Beyond the technical and ethical challenges, the increasing capabilities and adoption of generative AI also raise broader societal concerns:

*   **Erosion of Human Creativity:** Over-reliance on AI tools for content creation could potentially lead to an erosion of human creativity, skill development, and originality in creative and knowledge-based industries.
*   **Job Displacement:** There is a significant risk of job displacement in sectors heavily reliant on tasks that generative AI can automate or augment, particularly impacting roles involved in content creation, data analysis, and certain service industries.
*   **Safety and Security Risks:** Generative AI can amplify existing safety and security risks by providing threat actors with enhanced capabilities for executing sophisticated scams, fraud, phishing attacks, and large-scale disinformation campaigns.

**7. Responses and Mitigation Strategies**

In recognition of these challenges, there is a growing emphasis on proactive measures:

*   **Ethical AI Frameworks and Governance:** Significant efforts are directed towards developing robust ethical AI frameworks, guidelines, and establishing effective governance structures to guide the responsible development and deployment of generative AI.
*   **Bias Mitigation:** Active research and development are focused on techniques to identify and mitigate bias present in training data and within the AI models themselves, aiming for fairer and more equitable outputs.
*   **Transparency and Explainability:** Increasing transparency in how AI systems generate outputs and improving the explainability of their decision-making processes are key areas of focus to build trust and allow for verification.
*   **Energy Efficiency:** Efforts are underway to develop more energy-efficient AI models and optimize the underlying infrastructure to reduce the significant environmental impact of training and running large models.
*   **Workforce Development:** Organizations are recognizing the critical need to invest in reskilling and upskilling their workforce. This prepares employees to effectively collaborate with AI technologies, focusing on roles that leverage human unique strengths alongside AI capabilities.

**8. Conclusion**

As of early 2025, Generative AI is positioned in a dynamic state of rapid advancement and increasing integration across a diverse array of sectors. While it presents immense potential for driving innovation, achieving significant productivity gains, and enabling highly personalized experiences, it is simultaneously accompanied by substantial technical, ethical, and societal challenges. Successfully navigating these hurdles, including addressing data biases, managing "hallucinations," mitigating high costs and environmental impact, upholding ethical principles, and tackling societal implications, is paramount. Careful consideration and proactive mitigation strategies are essential for ensuring the responsible, equitable, and beneficial development and deployment of generative AI technologies for the future.

---

## EditorAgent

Okay, here is the quality-checked and slightly refined version of the report based on your request. The report was already well-structured and comprehensive, so the edits focus on minor improvements in phrasing, flow, and conciseness for maximum impact and clarity.

---

**Comprehensive Report: The Current State of Generative AI (as of early 2025)**

**Executive Summary**

As of early 2025, Generative Artificial Intelligence (AI) stands as a rapidly evolving and profoundly transformative technology. Moving beyond nascent stages, it is now being widely adopted across numerous industries, integrating into core business processes and consumer applications. Driven by advanced models like LLMs and increasingly multimodal capabilities, generative AI offers significant potential for innovation, personalization, and productivity gains. However, its rapid advancement is accompanied by substantial challenges, including data biases, "hallucinations," high computational costs, ethical and privacy concerns, and broader societal impacts such as job displacement and security risks. Addressing these limitations through robust ethical frameworks, effective governance, and focused mitigation strategies is crucial for realizing the technology's potential responsibly and equitably.

**1. Introduction to Generative AI**

Generative Artificial Intelligence is a sophisticated form of AI distinguished by its capacity to create *novel* content, a stark contrast to traditional AI which primarily focuses on analyzing or classifying *existing* data. This new content can manifest in various forms, including text, images, audio, code, and even 3D structures. The fundamental mechanism involves learning intricate patterns, structures, and distributions from vast datasets to produce outputs that are entirely new yet consistent with the training data's characteristics. This paradigm shift is largely propelled by significant advancements in underlying model architectures, particularly Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and most prominently, Large Language Models (LLMs).

**2. Current State of Adoption and Integration**

The current landscape of generative AI is characterized by widespread and rapidly accelerating adoption across both business and consumer domains. Organizations are transitioning beyond initial exploratory phases and are increasingly focused on the meaningful integration of generative AI into their core workflows and enterprise-level applications. This strategic shift is evidenced by substantial increases in investment dedicated to generative AI technologies across sectors. A key aspect of this adoption is the embedding of generative AI features directly into existing software and services. These features commonly appear as intuitive conversational interfaces, sophisticated digital assistants, helpful chatbots, and powerful "copilots" designed to augment human capabilities in various tasks, thereby boosting efficiency and unlocking new possibilities.

**3. Key Trends and Capabilities Shaping the Landscape**

Several key trends are defining the contemporary generative AI landscape:

*   **Rise of Multimodal AI:** A significant development is the emergence of advanced multimodal AI models. These models possess the capability to seamlessly process and generate content across multiple distinct modalities simultaneously, such as integrating text, images, audio, video, and even 3D data. This cross-modal understanding and generation capability vastly expands the range of potential applications for generative AI, moving beyond single-modality outputs.
*   **Hyper-Personalization:** Generative AI is enabling unprecedented levels of hyper-personalization across diverse industries. Sectors like marketing, e-commerce, healthcare, and gaming are leveraging generative models to tailor content, recommendations, and experiences precisely to the individual preferences, histories, and needs of users, creating highly engaging and relevant interactions.
*   **Real-Time Applications:** The increasing efficiency and reduced latency of generative models are leading to a greater prevalence of real-time generative applications. These applications facilitate dynamic tasks such as instant language translation during conversations, the rapid creation of dynamic content for live events or personalized marketing campaigns, and providing highly responsive and context-aware interactions in customer service and support scenarios.

**4. Future Outlook and Emerging Areas**

Looking towards the near future, several developments are anticipated to become more prominent:

*   **Autonomous AI Agents:** The emergence and increasing commonplace nature of autonomous AI agents powered by generative models are expected. These agents are designed with the capability to understand complex goals, break them down into tasks, execute those tasks, and make decisions autonomously or semi-autonomously, requiring minimal human intervention for certain functions.
*   **Domain-Specific/Vertical AI:** There is a discernible trend towards focusing generative AI applications on solving specific, complex problems within particular industries. This focus on "vertical AI" is accelerating the development of specialized generative models and applications tailored to the unique requirements, regulations, and proprietary data of sectors like healthcare (e.g., drug discovery, diagnostics), finance (e.g., fraud detection, market analysis), or manufacturing (e.g., design optimization, predictive maintenance).
*   **Hardware and Computational Advancements:** Continued advancements in hardware, particularly in specialized AI accelerators (like GPUs, TPUs) and optimized infrastructure, coupled with increasing computational power, are playing a crucial role. These advancements are contributing significantly to the rapid progress, enhanced capabilities, increased efficiency, and greater scalability of generative AI models, making previously infeasible applications possible.

**5. Significant Limitations and Challenges**

Despite its rapid advancements and transformative potential, generative AI currently faces several significant limitations and challenges that impede its flawless implementation and widespread trustworthy deployment:

*   **Data Reliance and Bias:** A major challenge stems from the technology's heavy reliance on the vast datasets used for training. Existing biases (societal, historical, representation) present within these datasets can be perpetuated and even amplified by the models, potentially leading to discriminatory, unfair, or irrelevant outputs, particularly affecting underrepresented groups or regions. Identifying and mitigating these biases is complex but critical.
*   **Hallucinations and Trustworthiness:** The phenomenon of "hallucinations," where models generate plausible-sounding but entirely inaccurate, fabricated, or nonsensical information, remains a critical issue. This directly impacts the reliability, factual accuracy, and trustworthiness of AI-generated content, necessitating careful verification, fact-checking, and human oversight, especially in sensitive applications.
*   **Lack of Truth Source and Reasoning:** Unlike humans, generative AI models inherently lack an independent "source of truth" or genuine world model. They often struggle with genuinely nuanced contextual understanding, causal relationships, and complex, multi-step reasoning, frequently relying on statistical patterns learned from data rather than true logical inference or verified facts.
*   **Computational Resources and Sustainability:** Training and running large generative models demand substantial computational resources and energy, leading to significant infrastructure costs and raising considerable concerns regarding environmental sustainability due to high carbon footprints associated with energy consumption.
*   **Ethical and Privacy Issues:** Paramount ethical and privacy concerns include the potential for accidental or malicious data leakage (of training or input data), the misuse of personal or sensitive data, a lack of transparency or "explainability" in how models arrive at their outputs, and the potential for generating harmful, deceptive, or misleading content such as sophisticated deepfakes, targeted propaganda, and widespread misinformation campaigns.
*   **Integration Difficulties:** Seamlessly integrating generative AI capabilities with diverse existing legacy systems, complex enterprise workflows, and organizational data structures can be technically challenging and costly, posing a particular barrier for small and medium-sized enterprises (SMEs) with limited technical expertise and resources.
*   **Intellectual Property Concerns:** Navigating intellectual property rights (IPR) is complex, raising significant legal and ethical questions about the ownership of AI-generated content and the potential for generative models to inadvertently infringe on copyrighted material used, sometimes without explicit consent, in their training data or incorporated into their output generation.
*   **Language Barriers:** While progress is being made, performance disparities and limitations often persist between different languages, dialects, and cultural nuances. Models typically perform best in languages heavily represented in training data (primarily English), leading to less reliable or culturally inappropriate outputs in other linguistic contexts.

**6. Broader Societal Concerns**

Beyond the technical and ethical challenges, the increasing capabilities and widespread adoption of generative AI also raise broader societal concerns:

*   **Erosion of Human Creativity and Skills:** Over-reliance on AI tools for tasks like writing, design, and coding could potentially lead to an erosion of fundamental human creativity, critical thinking skills, skill development, and originality in creative and knowledge-based industries if not used as augmentation tools.
*   **Job Displacement and Workforce Transformation:** There is a significant risk of job displacement in sectors heavily reliant on tasks that generative AI can automate or significantly augment, particularly impacting roles involved in content creation, data analysis, customer service, and certain administrative functions. This necessitates a focus on workforce transformation and adaptation.
*   **Safety and Security Risks:** Generative AI can amplify existing safety and security risks by providing threat actors with enhanced capabilities for executing sophisticated scams, phishing attacks, fraud, identity theft, automated cyberattacks, and large-scale, highly convincing disinformation campaigns designed to manipulate public opinion or sow discord.

**7. Responses and Mitigation Strategies**

In recognition of these challenges and concerns, there is a growing emphasis on proactive measures from researchers, developers, policymakers, and organizations:

*   **Ethical AI Frameworks and Governance:** Significant efforts are directed towards developing robust ethical AI frameworks, industry guidelines, best practices, and establishing effective national and international governance structures and regulations to guide the responsible development, deployment, and use of generative AI, prioritizing human well-being and societal benefit.
*   **Bias Mitigation and Fairness:** Active research and development are focused on techniques to identify, measure, and mitigate bias present in training data and within the AI models themselves, aiming for fairer, more equitable, and less discriminatory outputs across diverse populations.
*   **Transparency, Explainability, and Watermarking:** Increasing transparency in how AI systems generate outputs ("explainable AI" or XAI) and exploring methods for watermarking or digitally signing AI-generated content are key areas of focus to build trust, allow for verification, and distinguish AI outputs from human creations.
*   **Energy Efficiency and Optimization:** Efforts are underway to develop more energy-efficient AI model architectures, optimize training algorithms, and improve the underlying hardware and data center infrastructure to reduce the significant environmental impact and operational costs of large-scale AI deployment.
*   **Workforce Development and Adaptation:** Organizations and governments are recognizing the critical need to invest heavily in reskilling and upskilling their workforce. This prepares employees to effectively collaborate *with* AI technologies, focusing on developing skills that leverage human unique strengths (creativity, critical judgment, empathy) alongside AI capabilities, fostering an AI-augmented future of work.

**8. Conclusion**

As of early 2025, Generative AI is positioned in a dynamic state of rapid advancement and increasing integration across a diverse array of sectors globally. While it presents immense potential for driving innovation, achieving significant productivity gains, enabling highly personalized experiences, and solving complex problems, it is simultaneously accompanied by substantial technical, ethical, and societal challenges. Successfully navigating these hurdles, including addressing data biases, managing "hallucinations" and reliability, mitigating high costs and environmental impact, upholding ethical principles and privacy, ensuring security, and tackling societal implications like job displacement and misinformation, is paramount. Careful consideration, collaborative effort across stakeholders, and proactive mitigation strategies are essential for ensuring the responsible, equitable, secure, and ultimately beneficial development and deployment of generative AI technologies for the future.

---

## Parallel Agents

Sometimes, running agents in parallel is desirable, particularly for tasks like translation. The Agent Developer Kit supports such parallel orchestration. In this session, you will create a parallel pipeline designed to translate a report into several languages simultaneously.

In [15]:
# Define variables
APP_NAME = "parallel_app"
USER_ID = "parallel_user"
SESSION_ID = "parallel_session"

#### Initialize the agents and pipeline

In [16]:
# Define respective agents and pipeline
bahasa_translator = LlmAgent(
    name="BahasaAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Bahasa lanaguage",
    description="Translate a given report to Bahasa",
    output_key="bahasa_version"
)

thai_translator = LlmAgent(
    name="ThaiAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Thai lanaguage",
    description="Translate a given report to Thai",
    output_key="thai_version"
)

vietnamese_translator = LlmAgent(
    name="VietnameseAgent",
    model=GEMINI_MODEL_NAME,
    instruction="Your task is to translate the given report into Vietnamese lanaguage",
    description="Translate a given report to Vietnamese",
    output_key="vietnam_version"
)

parallel_pipeline = ParallelAgent(
    name="TranslationPipelineAgent",
    sub_agents=[bahasa_translator, thai_translator, vietnamese_translator],
    description="Runs multiple tranlsation agents in parallel to translate the report.",
)

#### Initialize Session & Runner

In [17]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Initialize Runner
parallel_runner = Runner(
    agent=parallel_pipeline,
    app_name=APP_NAME,
    session_service=session_service,
)

In [21]:
prompt = f"translate this report: {report}"
output = await run_agent(parallel_runner, USER_ID, SESSION_ID, prompt)

## BahasaAgent

Tentu, ini terjemahan laporan tersebut ke dalam Bahasa Indonesia:

---

**Laporan Komprehensif: Keadaan Generative AI Saat Ini (per Awal 2025)**

**Ringkasan Eksekutif**

Per awal tahun 2025, Kecerdasan Buatan Generatif (Generative Artificial Intelligence/AI) berdiri sebagai teknologi yang berkembang pesat dan sangat transformatif. Melampaui tahap awal, kini teknologi ini telah banyak diadopsi di berbagai industri, terintegrasi ke dalam proses bisnis inti dan aplikasi konsumen. Didorong oleh model canggih seperti LLM dan kapabilitas yang semakin multimodal, AI generatif menawarkan potensi signifikan untuk inovasi, personalisasi, dan peningkatan produktivitas. Namun, kemajuannya yang pesat juga disertai dengan tantangan substansial, termasuk bias data, "halusinasi" (generasi informasi salah), biaya komputasi yang tinggi, masalah etika dan privasi, serta dampak sosial yang lebih luas seperti pergeseran pekerjaan dan risiko keamanan. Menangani keterbatasan ini melalui kerangka kerja etika yang kuat, tata kelola yang efektif, dan strategi mitigasi yang terfokus sangat penting untuk mewujudkan potensi teknologi secara bertanggung jawab dan adil.

**1. Pengantar Generative AI**

Kecerdasan Buatan Generatif adalah bentuk AI yang canggih yang dibedakan oleh kemampuannya untuk menciptakan konten *baru*, berbeda sekali dengan AI tradisional yang terutama berfokus pada analisis atau klasifikasi data *yang sudah ada*. Konten baru ini dapat bermanifestasi dalam berbagai bentuk, termasuk teks, gambar, audio, kode, dan bahkan struktur 3D. Mekanisme fundamentalnya melibatkan pembelajaran pola, struktur, dan distribusi yang rumit dari kumpulan data besar untuk menghasilkan output yang sepenuhnya baru namun konsisten dengan karakteristik data pelatihan. Pergeseran paradigma ini sebagian besar didorong oleh kemajuan signifikan dalam arsitektur model dasar, terutama Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), dan yang paling menonjol, Large Language Models (LLMs).

**2. Keadaan Adopsi dan Integrasi Saat Ini**

Lanskap AI generatif saat ini dicirikan oleh adopsi yang meluas dan berkembang pesat di domain bisnis maupun konsumen. Organisasi beralih melampaui fase eksplorasi awal dan semakin berfokus pada integrasi AI generatif yang berarti ke dalam alur kerja inti dan aplikasi tingkat perusahaan mereka. Pergeseran strategis ini dibuktikan dengan peningkatan substansial dalam investasi yang didedikasikan untuk teknologi AI generatif di berbagai sektor. Aspek kunci dari adopsi ini adalah penyematan fitur AI generatif secara langsung ke dalam perangkat lunak dan layanan yang sudah ada. Fitur-fitur ini umumnya muncul sebagai antarmuka percakapan yang intuitif, asisten digital yang canggih, chatbot yang membantu, dan "copilot" yang kuat yang dirancang untuk meningkatkan kapabilitas manusia dalam berbagai tugas, sehingga meningkatkan efisiensi dan membuka kemungkinan baru.

**3. Tren Utama dan Kapabilitas yang Membentuk Lanskap**

Beberapa tren utama mendefinisikan lanskap AI generatif kontemporer:

*   **Kebangkitan Multimodal AI:** Perkembangan signifikan adalah munculnya model AI multimodal canggih. Model-model ini memiliki kemampuan untuk memproses dan menghasilkan konten dengan lancar di berbagai modalitas yang berbeda secara bersamaan, seperti mengintegrasikan data teks, gambar, audio, video, dan bahkan 3D. Kemampuan pemahaman dan generasi lintas modal ini secara luas memperluas jangkauan potensi aplikasi untuk AI generatif, melampaui output modalitas tunggal.
*   **Hiper-Personalisasi:** AI generatif memungkinkan tingkat hiper-personalisasi yang belum pernah terjadi sebelumnya di berbagai industri. Sektor seperti pemasaran, e-commerce, perawatan kesehatan, dan gaming memanfaatkan model generatif untuk menyesuaikan konten, rekomendasi, dan pengalaman secara tepat dengan preferensi, riwayat, dan kebutuhan individu pengguna, menciptakan interaksi yang sangat menarik dan relevan.
*   **Aplikasi Real-Time:** Peningkatan efisiensi dan pengurangan latensi model generatif mengarah pada prevalensi aplikasi generatif real-time yang lebih besar. Aplikasi ini memfasilitasi tugas-tugas dinamis seperti terjemahan bahasa instan selama percakapan, pembuatan konten dinamis yang cepat untuk acara langsung atau kampanye pemasaran yang dipersonalisasi, dan penyediaan interaksi yang sangat responsif dan sadar konteks dalam skenario layanan dan dukungan pelanggan.

**4. Prospek Masa Depan dan Area yang Berkembang**

Melihat ke masa depan dekat, beberapa perkembangan diperkirakan akan menjadi lebih menonjol:

*   **Agen AI Otonom:** Munculnya dan semakin umumnya agen AI otonom yang didukung oleh model generatif diperkirakan akan terjadi. Agen-agen ini dirancang dengan kemampuan untuk memahami tujuan yang kompleks, memecahnya menjadi tugas-tugas, menjalankan tugas-tugas tersebut, dan membuat keputusan secara otonom atau semi-otonom, memerlukan intervensi manusia minimal untuk fungsi-fungsi tertentu.
*   **AI Spesifik Domain/Vertikal:** Ada tren yang jelas menuju fokus aplikasi AI generatif pada pemecahan masalah spesifik dan kompleks dalam industri tertentu. Fokus pada "AI vertikal" ini mempercepat pengembangan model dan aplikasi generatif khusus yang disesuaikan dengan persyaratan unik, regulasi, dan data kepemilikan sektor seperti perawatan kesehatan (misalnya, penemuan obat, diagnostik), keuangan (misalnya, deteksi penipuan, analisis pasar), atau manufaktur (misalnya, optimasi desain, pemeliharaan prediktif).
*   **Kemajuan Perangkat Keras dan Komputasi:** Kemajuan berkelanjutan dalam perangkat keras, terutama dalam akselerator AI khusus (seperti GPU, TPU) dan infrastruktur yang dioptimalkan, ditambah dengan peningkatan daya komputasi, memainkan peran penting. Kemajuan ini berkontribusi secara signifikan terhadap kemajuan pesat, kapabilitas yang ditingkatkan, efisiensi yang meningkat, dan skalabilitas model AI generatif yang lebih besar, memungkinkan aplikasi yang sebelumnya tidak dapat dilakukan.

**5. Keterbatasan dan Tantangan Signifikan**

Terlepas dari kemajuannya yang pesat dan potensi transformatifnya, AI generatif saat ini menghadapi beberapa keterbatasan dan tantangan signifikan yang menghambat implementasi sempurna dan penyebaran tepercaya secara luas:

*   **Ketergantungan dan Bias Data:** Tantangan utama berasal dari ketergantungan teknologi yang besar pada kumpulan data luas yang digunakan untuk pelatihan. Bias yang ada (sosial, historis, representasi) yang terdapat dalam kumpulan data ini dapat diabadikan dan bahkan diperkuat oleh model, berpotensi menghasilkan output yang diskriminatif, tidak adil, atau tidak relevan, terutama memengaruhi kelompok atau wilayah yang kurang terwakili. Mengidentifikasi dan memitigasi bias ini rumit namun sangat penting.
*   **Halusinasi dan Kepercayaan:** Fenomena "halusinasi", di mana model menghasilkan informasi yang terdengar masuk akal tetapi sepenuhnya tidak akurat, mengada-ada, atau tidak masuk akal, tetap menjadi masalah kritis. Ini secara langsung memengaruhi keandalan, akurasi faktual, dan kepercayaan konten yang dihasilkan AI, memerlukan verifikasi, pemeriksaan fakta, dan pengawasan manusia yang cermat, terutama dalam aplikasi sensitif.
*   **Kurangnya Sumber Kebenaran dan Penalaran:** Berbeda dengan manusia, model AI generatif secara inheren kekurangan "sumber kebenaran" independen atau model dunia yang asli. Mereka sering kali kesulitan dengan pemahaman kontekstual yang benar-benar bernuansa, hubungan sebab akibat, dan penalaran multi-langkah yang kompleks, sering kali mengandalkan pola statistik yang dipelajari dari data daripada inferensi logis yang sebenarnya atau fakta yang diverifikasi.
*   **Sumber Daya Komputasi dan Keberlanjutan:** Melatih dan menjalankan model generatif besar membutuhkan sumber daya komputasi dan energi yang substansial, menyebabkan biaya infrastruktur yang signifikan dan menimbulkan kekhawatiran yang cukup besar mengenai keberlanjutan lingkungan karena jejak karbon yang tinggi terkait dengan konsumsi energi.
*   **Masalah Etika dan Privasi:** Kekhawatiran etika dan privasi yang paling utama meliputi potensi kebocoran data yang tidak disengaja atau berbahaya (data pelatihan atau input), penyalahgunaan data pribadi atau sensitif, kurangnya transparansi atau "kemampuan menjelaskan" (explainability) dalam cara model mencapai output mereka, dan potensi untuk menghasilkan konten yang berbahaya, menipu, atau menyesatkan seperti deepfake canggih, propaganda yang ditargetkan, dan kampanye misinformasi berskala besar.
*   **Kesulitan Integrasi:** Mengintegrasikan kapabilitas AI generatif dengan lancar ke dalam sistem lama yang beragam, alur kerja perusahaan yang kompleks, dan struktur data organisasi bisa menjadi tantangan teknis dan mahal, menjadi penghalang khusus bagi usaha kecil dan menengah (UKM) dengan keahlian teknis dan sumber daya yang terbatas.
*   **Kekhawatiran Kekayaan Intelektual:** Menavigasi hak kekayaan intelektual (HKI) kompleks, menimbulkan pertanyaan hukum dan etika yang signifikan tentang kepemilikan konten yang dihasilkan AI dan potensi model generatif untuk secara tidak sengaja melanggar materi berhak cipta yang digunakan, terkadang tanpa persetujuan eksplisit, dalam data pelatihan mereka atau dimasukkan ke dalam generasi output mereka.
*   **Hambatan Bahasa:** Meskipun kemajuan sedang dibuat, perbedaan kinerja dan keterbatasan sering kali tetap ada antara berbagai bahasa, dialek, dan nuansa budaya. Model biasanya berkinerja terbaik dalam bahasa yang sangat terwakili dalam data pelatihan (terutama Bahasa Inggris), yang mengarah pada output yang kurang dapat diandalkan atau tidak sesuai secara budaya dalam konteks linguistik lainnya.

**6. Kekhawatiran Sosial yang Lebih Luas**

Selain tantangan teknis dan etika, peningkatan kapabilitas dan adopsi AI generatif yang meluas juga menimbulkan kekhawatiran sosial yang lebih luas:

*   **Erosi Kreativitas dan Keterampilan Manusia:** Ketergantungan berlebihan pada alat AI untuk tugas-tugas seperti menulis, mendesain, dan coding berpotensi menyebabkan erosi kreativitas fundamental manusia, keterampilan berpikir kritis, pengembangan keterampilan, dan orisinalitas dalam industri kreatif dan berbasis pengetahuan jika tidak digunakan sebagai alat bantu peningkatan.
*   **Pergeseran Pekerjaan dan Transformasi Tenaga Kerja:** Ada risiko signifikan pergeseran pekerjaan di sektor-sektor yang sangat bergantung pada tugas-tugas yang dapat diotomatisasi atau ditingkatkan secara signifikan oleh AI generatif, terutama memengaruhi peran yang melibatkan pembuatan konten, analisis data, layanan pelanggan, dan fungsi administratif tertentu. Hal ini memerlukan fokus pada transformasi dan adaptasi tenaga kerja.
*   **Risiko Keselamatan dan Keamanan:** AI generatif dapat memperkuat risiko keselamatan dan keamanan yang ada dengan memberi para pelaku ancaman kemampuan yang ditingkatkan untuk melaksanakan penipuan canggih, serangan phishing, penipuan, pencurian identitas, serangan siber otomatis, dan kampanye disinformasi berskala besar yang sangat meyakinkan yang dirancang untuk memanipulasi opini publik atau menabur perpecahan.

**7. Tanggapan dan Strategi Mitigasi**

Dalam pengakuan atas tantangan dan kekhawatiran ini, ada penekanan yang meningkat pada langkah-langkah proaktif dari peneliti, pengembang, pembuat kebijakan, dan organisasi:

*   **Kerangka Kerja Etika AI dan Tata Kelola:** Upaya signifikan diarahkan untuk mengembangkan kerangka kerja etika AI yang kuat, pedoman industri, praktik terbaik, dan membangun struktur tata kelola dan regulasi nasional dan internasional yang efektif untuk memandu pengembangan, penyebaran, dan penggunaan AI generatif yang bertanggung jawab, memprioritaskan kesejahteraan manusia dan manfaat sosial.
*   **Mitigasi Bias dan Keadilan:** Penelitian dan pengembangan aktif difokuskan pada teknik untuk mengidentifikasi, mengukur, dan memitigasi bias yang ada dalam data pelatihan dan di dalam model AI itu sendiri, bertujuan untuk output yang lebih adil, lebih setara, dan kurang diskriminatif di berbagai populasi.
*   **Transparansi, Kemampuan Menjelaskan, dan Watermarking:** Meningkatkan transparansi dalam cara sistem AI menghasilkan output ("explainable AI" atau XAI) dan mengeksplorasi metode watermarking atau penandatanganan digital pada konten yang dihasilkan AI adalah area fokus utama untuk membangun kepercayaan, memungkinkan verifikasi, dan membedakan output AI dari kreasi manusia.
*   **Efisiensi Energi dan Optimasi:** Upaya sedang dilakukan untuk mengembangkan arsitektur model AI yang lebih hemat energi, mengoptimalkan algoritma pelatihan, dan meningkatkan perangkat keras dan infrastruktur pusat data yang mendasarinya untuk mengurangi dampak lingkungan yang signifikan dan biaya operasional dari penyebaran AI berskala besar.
*   **Pengembangan dan Adaptasi Tenaga Kerja:** Organisasi dan pemerintah menyadari kebutuhan kritis untuk berinvestasi besar-besaran dalam melatih ulang (reskilling) dan meningkatkan keterampilan (upskilling) tenaga kerja mereka. Hal ini mempersiapkan karyawan untuk berkolaborasi secara efektif *dengan* teknologi AI, berfokus pada pengembangan keterampilan yang memanfaatkan kekuatan unik manusia (kreativitas, penilaian kritis, empati) bersama kapabilitas AI, menumbuhkan masa depan kerja yang diperkuat AI.

**8. Kesimpulan**

Per awal tahun 2025, Generative AI berada dalam keadaan dinamis kemajuan pesat dan peningkatan integrasi di beragam sektor secara global. Meskipun menawarkan potensi besar untuk mendorong inovasi, mencapai peningkatan produktivitas yang signifikan, memungkinkan pengalaman yang sangat dipersonalisasi, dan memecahkan masalah yang kompleks, pada saat yang sama juga disertai dengan tantangan teknis, etika, dan sosial yang substansial. Berhasil menavigasi rintangan ini, termasuk mengatasi bias data, mengelola "halusinasi" dan keandalan, memitigasi biaya tinggi dan dampak lingkungan, menjunjung tinggi prinsip etika dan privasi, memastikan keamanan, dan menangani implikasi sosial seperti pergeseran pekerjaan dan misinformasi, adalah hal yang sangat penting. Pertimbangan yang cermat, upaya kolaboratif di antara para pemangku kepentingan, dan strategi mitigasi proaktif sangat penting untuk memastikan pengembangan dan penyebaran teknologi AI generatif yang bertanggung jawab, adil, aman, dan pada akhirnya bermanfaat untuk masa depan.

---

## VietnamAgent

Tuyệt vời, đây là phiên bản báo cáo đã được kiểm tra chất lượng và tinh chỉnh một chút dựa trên yêu cầu của bạn. Báo cáo gốc đã có cấu trúc tốt và toàn diện, vì vậy các chỉnh sửa tập trung vào những cải thiện nhỏ về cách diễn đạt, mạch lạc và súc tích để đạt hiệu quả và sự rõ ràng tối đa.

---

**Báo Cáo Toàn Diện: Hiện Trạng Trí Tuệ Nhân Tạo Tạo Sinh (Tính đến đầu năm 2025)**

**Tóm Tắt Điều Hành**

Tính đến đầu năm 2025, Trí tuệ Nhân tạo Tạo sinh (AI tạo sinh) đang là một công nghệ phát triển nhanh chóng và có khả năng chuyển đổi sâu sắc. Vượt ra khỏi giai đoạn sơ khai, công nghệ này hiện đang được ứng dụng rộng rãi trên nhiều ngành công nghiệp, tích hợp vào các quy trình kinh doanh cốt lõi và ứng dụng tiêu dùng. Được thúc đẩy bởi các mô hình tiên tiến như Mô hình Ngôn ngữ Lớn (LLM) và khả năng ngày càng đa phương thức, AI tạo sinh mang đến tiềm năng đáng kể cho sự đổi mới, cá nhân hóa và tăng năng suất. Tuy nhiên, sự tiến bộ nhanh chóng của nó cũng đi kèm với những thách thức đáng kể, bao gồm thiên vị dữ liệu, "ảo giác", chi phí tính toán cao, lo ngại về đạo đức và quyền riêng tư, cùng với những tác động xã hội rộng lớn hơn như thay thế việc làm và rủi ro an ninh. Việc giải quyết những hạn chế này thông qua các khuôn khổ đạo đức vững chắc, quản trị hiệu quả và các chiến lược giảm thiểu tập trung là điều tối quan trọng để hiện thực hóa tiềm năng của công nghệ một cách có trách nhiệm và công bằng.

**1. Giới thiệu về Trí tuệ Nhân tạo Tạo sinh**

Trí tuệ Nhân tạo Tạo sinh là một dạng AI tinh vi, được phân biệt bởi khả năng tạo ra nội dung *mới lạ*, khác biệt rõ rệt với AI truyền thống vốn chủ yếu tập trung vào phân tích hoặc phân loại *dữ liệu hiện có*. Nội dung mới này có thể biểu hiện dưới nhiều hình thức khác nhau, bao gồm văn bản, hình ảnh, âm thanh, mã code, và thậm chí cả cấu trúc 3D. Cơ chế cơ bản liên quan đến việc học các mẫu, cấu trúc và phân bố phức tạp từ các bộ dữ liệu khổng lồ để tạo ra các đầu ra hoàn toàn mới nhưng nhất quán với đặc điểm của dữ liệu huấn luyện. Sự thay đổi mô hình này phần lớn được thúc đẩy bởi những tiến bộ đáng kể trong kiến trúc mô hình nền tảng, đặc biệt là Mạng đối nghịch tạo sinh (GANs), Bộ tự mã hóa biến phân (VAEs), và nổi bật nhất là Mô hình ngôn ngữ lớn (LLMs).

**2. Hiện trạng ứng dụng và tích hợp**

Bối cảnh hiện tại của AI tạo sinh được đặc trưng bởi việc ứng dụng rộng rãi và tăng tốc nhanh chóng trên cả lĩnh vực doanh nghiệp và tiêu dùng. Các tổ chức đang chuyển đổi vượt ra ngoài các giai đoạn thăm dò ban đầu và ngày càng tập trung vào việc tích hợp có ý nghĩa AI tạo sinh vào các quy trình làm việc cốt lõi và ứng dụng cấp doanh nghiệp của họ. Sự thay đổi chiến lược này được chứng minh bằng sự gia tăng đáng kể về đầu tư dành cho các công nghệ AI tạo sinh trên các ngành. Một khía cạnh quan trọng của việc ứng dụng này là việc tích hợp các tính năng AI tạo sinh trực tiếp vào phần mềm và dịch vụ hiện có. Các tính năng này thường xuất hiện dưới dạng giao diện đàm thoại trực quan, trợ lý kỹ thuật số tinh vi, chatbot hữu ích và "copilot" mạnh mẽ được thiết kế để tăng cường khả năng của con người trong nhiều nhiệm vụ khác nhau, từ đó nâng cao hiệu quả và mở khóa những khả năng mới.

**3. Các xu hướng chính và khả năng định hình bối cảnh**

Một số xu hướng chính đang định hình bối cảnh AI tạo sinh hiện tại:

*   **Sự trỗi dậy của AI đa phương thức:** Một sự phát triển quan trọng là sự xuất hiện của các mô hình AI đa phương thức tiên tiến. Các mô hình này sở hữu khả năng xử lý và tạo nội dung liền mạch trên nhiều phương thức khác nhau cùng lúc, chẳng hạn như tích hợp văn bản, hình ảnh, âm thanh, video và thậm chí cả dữ liệu 3D. Khả năng hiểu và tạo nội dung đa phương thức này mở rộng đáng kể phạm vi ứng dụng tiềm năng cho AI tạo sinh, vượt ra ngoài các đầu ra đơn phương thức.
*   **Siêu cá nhân hóa:** AI tạo sinh đang cho phép mức độ siêu cá nhân hóa chưa từng có trên các ngành khác nhau. Các lĩnh vực như tiếp thị, thương mại điện tử, chăm sóc sức khỏe và trò chơi đang tận dụng các mô hình tạo sinh để điều chỉnh nội dung, đề xuất và trải nghiệm chính xác theo sở thích, lịch sử và nhu cầu cá nhân của người dùng, tạo ra các tương tác có tính tương tác cao và phù hợp.
*   **Ứng dụng thời gian thực:** Hiệu quả tăng lên và độ trễ giảm của các mô hình tạo sinh đang dẫn đến sự phổ biến lớn hơn của các ứng dụng tạo sinh thời gian thực. Các ứng dụng này tạo điều kiện thuận lợi cho các tác vụ động như dịch ngôn ngữ tức thời trong các cuộc hội thoại, nhanh chóng tạo nội dung động cho các sự kiện trực tiếp hoặc chiến dịch tiếp thị được cá nhân hóa, và cung cấp các tương tác phản hồi nhanh và nhận biết ngữ cảnh trong các tình huống dịch vụ và hỗ trợ khách hàng.

**4. Triển vọng tương lai và các lĩnh vực mới nổi**

Hướng tới tương lai gần, một số sự phát triển dự kiến sẽ trở nên nổi bật hơn:

*   **Các tác nhân AI tự hành:** Sự xuất hiện và tính phổ biến ngày càng tăng của các tác nhân AI tự hành được hỗ trợ bởi các mô hình tạo sinh được kỳ vọng. Các tác nhân này được thiết kế với khả năng hiểu các mục tiêu phức tạp, chia nhỏ thành các nhiệm vụ, thực hiện các nhiệm vụ đó và đưa ra quyết định một cách tự hành hoặc bán tự hành, đòi hỏi sự can thiệp tối thiểu của con người đối với một số chức năng nhất định.
*   **AI chuyên ngành/theo chiều dọc:** Có một xu hướng rõ ràng hướng tới việc tập trung các ứng dụng AI tạo sinh vào việc giải quyết các vấn đề cụ thể, phức tạp trong các ngành công nghiệp cụ thể. Sự tập trung vào "AI theo chiều dọc" này đang thúc đẩy sự phát triển của các mô hình và ứng dụng tạo sinh chuyên biệt phù hợp với các yêu cầu, quy định và dữ liệu độc quyền riêng biệt của các lĩnh vực như chăm sóc sức khỏe (ví dụ: khám phá thuốc, chẩn đoán), tài chính (ví dụ: phát hiện gian lận, phân tích thị trường) hoặc sản xuất (ví dụ: tối ưu hóa thiết kế, bảo trì dự đoán).
*   **Những tiến bộ về phần cứng và tính toán:** Những tiến bộ liên tục về phần cứng, đặc biệt là trong các bộ tăng tốc AI chuyên dụng (như GPU, TPU) và cơ sở hạ tầng tối ưu, cùng với sức mạnh tính toán ngày càng tăng, đang đóng vai trò quan trọng. Những tiến bộ này đóng góp đáng kể vào sự tiến bộ nhanh chóng, khả năng nâng cao, hiệu quả tăng lên và khả năng mở rộng lớn hơn của các mô hình AI tạo sinh, giúp các ứng dụng trước đây không khả thi trở nên khả thi.

**5. Những hạn chế và thách thức đáng kể**

Mặc dù có những tiến bộ nhanh chóng và tiềm năng chuyển đổi, AI tạo sinh hiện đang đối mặt với một số hạn chế và thách thức đáng kể cản trở việc triển khai hoàn hảo và ứng dụng rộng rãi một cách đáng tin cậy:

*   **Sự phụ thuộc vào dữ liệu và thiên vị:** Một thách thức lớn xuất phát từ sự phụ thuộc nặng nề của công nghệ vào các bộ dữ liệu khổng lồ được sử dụng để huấn luyện. Những thiên vị hiện có (xã hội, lịch sử, đại diện) tồn tại trong các bộ dữ liệu này có thể bị duy trì và thậm chí khuếch đại bởi các mô hình, có khả năng dẫn đến các đầu ra mang tính phân biệt đối xử, không công bằng hoặc không phù hợp, đặc biệt ảnh hưởng đến các nhóm hoặc khu vực ít được đại diện. Việc xác định và giảm thiểu những thiên vị này là phức tạp nhưng rất quan trọng.
*   **Ảo giác và độ tin cậy:** Hiện tượng "ảo giác", nơi các mô hình tạo ra thông tin nghe có vẻ hợp lý nhưng hoàn toàn không chính xác, bịa đặt hoặc vô nghĩa, vẫn là một vấn đề nghiêm trọng. Điều này ảnh hưởng trực tiếp đến độ tin cậy, tính chính xác về mặt thực tế và độ tin cậy của nội dung do AI tạo ra, đòi hỏi phải kiểm tra cẩn thận, xác minh sự thật và sự giám sát của con người, đặc biệt là trong các ứng dụng nhạy cảm.
*   **Thiếu nguồn sự thật và khả năng suy luận:** Không giống như con người, các mô hình AI tạo sinh vốn dĩ thiếu "nguồn sự thật" độc lập hoặc mô hình thế giới thực. Chúng thường gặp khó khăn trong việc hiểu ngữ cảnh phức tạp thực sự, mối quan hệ nhân quả và suy luận phức tạp, nhiều bước, thường dựa vào các mẫu thống kê học được từ dữ liệu thay vì suy luận logic thực sự hoặc sự thật đã được xác minh.
*   **Tài nguyên tính toán và tính bền vững:** Huấn luyện và chạy các mô hình tạo sinh lớn đòi hỏi tài nguyên tính toán và năng lượng đáng kể, dẫn đến chi phí cơ sở hạ tầng cao và gây ra những lo ngại đáng kể về tính bền vững môi trường do lượng khí thải carbon cao liên quan đến tiêu thụ năng lượng.
*   **Vấn đề đạo đức và quyền riêng tư:** Những lo ngại hàng đầu về đạo đức và quyền riêng tư bao gồm nguy cơ rò rỉ dữ liệu do tai nạn hoặc cố ý (của dữ liệu huấn luyện hoặc dữ liệu đầu vào), lạm dụng dữ liệu cá nhân hoặc nhạy cảm, thiếu minh bạch hoặc "khả năng giải thích" trong cách các mô hình đưa ra đầu ra của chúng, và tiềm năng tạo ra nội dung có hại, lừa đảo hoặc gây hiểu lầm như deepfake tinh vi, tuyên truyền có mục tiêu và các chiến dịch thông tin sai lệch quy mô lớn được thiết kế để thao túng dư luận hoặc gieo rắc bất hòa.
*   **Khó khăn trong việc tích hợp:** Việc tích hợp liền mạch các khả năng AI tạo sinh với các hệ thống kế thừa đa dạng hiện có, quy trình làm việc phức tạp của doanh nghiệp và cấu trúc dữ liệu của tổ chức có thể gặp thách thức về kỹ thuật và tốn kém, tạo ra một rào cản đặc biệt đối với các doanh nghiệp nhỏ và vừa (SME) với kiến thức chuyên môn và tài nguyên kỹ thuật hạn chế.
*   **Các lo ngại về sở hữu trí tuệ:** Điều hướng quyền sở hữu trí tuệ (IPR) rất phức tạp, đặt ra những câu hỏi pháp lý và đạo đức đáng kể về quyền sở hữu nội dung do AI tạo ra và khả năng các mô hình tạo sinh vô tình vi phạm tài liệu có bản quyền được sử dụng, đôi khi không có sự đồng ý rõ ràng, trong dữ liệu huấn luyện của chúng hoặc được đưa vào quá trình tạo đầu ra của chúng.
*   **Rào cản ngôn ngữ:** Mặc dù đang có tiến bộ, sự chênh lệch về hiệu suất và những hạn chế thường tồn tại giữa các ngôn ngữ, phương ngữ và sắc thái văn hóa khác nhau. Các mô hình thường hoạt động tốt nhất ở các ngôn ngữ được thể hiện nhiều trong dữ liệu huấn luyện (chủ yếu là tiếng Anh), dẫn đến các đầu ra kém tin cậy hơn hoặc không phù hợp về mặt văn hóa trong các ngữ cảnh ngôn ngữ khác.

**6. Những lo ngại xã hội rộng hơn**

Ngoài những thách thức kỹ thuật và đạo đức, khả năng ngày càng tăng và việc ứng dụng rộng rãi AI tạo sinh cũng đặt ra những lo ngại xã hội rộng hơn:

*   **Xói mòn sự sáng tạo và kỹ năng của con người:** Sự phụ thuộc quá mức vào các công cụ AI cho các tác vụ như viết, thiết kế và viết mã có thể dẫn đến sự xói mòn tiềm tàng về sự sáng tạo cơ bản của con người, kỹ năng tư duy phản biện, phát triển kỹ năng và tính độc đáo trong các ngành công nghiệp sáng tạo và dựa trên tri thức nếu không được sử dụng như công cụ tăng cường.
*   **Thay thế việc làm và chuyển đổi lực lượng lao động:** Có nguy cơ đáng kể về việc thay thế việc làm trong các lĩnh vực phụ thuộc nhiều vào các tác vụ mà AI tạo sinh có thể tự động hóa hoặc tăng cường đáng kể, đặc biệt ảnh hưởng đến các vai trò liên quan đến tạo nội dung, phân tích dữ liệu, dịch vụ khách hàng và một số chức năng hành chính. Điều này đòi hỏi phải tập trung vào chuyển đổi và thích ứng lực lượng lao động.
*   **Nguy cơ về an toàn và bảo mật:** AI tạo sinh có thể khuếch đại các nguy cơ về an toàn và bảo mật hiện có bằng cách cung cấp cho các tác nhân đe dọa những khả năng nâng cao để thực hiện các vụ lừa đảo tinh vi, tấn công lừa đảo (phishing), gian lận, đánh cắp danh tính, tấn công mạng tự động và các chiến dịch thông tin sai lệch quy mô lớn, rất thuyết phục được thiết kế để thao túng dư luận hoặc gieo rắc bất hòa.

**7. Các biện pháp ứng phó và chiến lược giảm thiểu**

Nhận thức được những thách thức và lo ngại này, ngày càng có sự chú trọng vào các biện pháp chủ động từ các nhà nghiên cứu, nhà phát triển, nhà hoạch định chính sách và các tổ chức:

*   **Khuôn khổ đạo đức AI và quản trị:** Những nỗ lực đáng kể được định hướng tới việc phát triển các khuôn khổ đạo đức AI vững chắc, hướng dẫn ngành, thực tiễn tốt nhất, và thiết lập cấu trúc quản trị và quy định quốc gia và quốc tế hiệu quả để hướng dẫn việc phát triển, triển khai và sử dụng AI tạo sinh một cách có trách nhiệm, ưu tiên hạnh phúc của con người và lợi ích xã hội.
*   **Giảm thiểu thiên vị và đảm bảo công bằng:** Nghiên cứu và phát triển tích cực đang tập trung vào các kỹ thuật để xác định, đo lường và giảm thiểu thiên vị tồn tại trong dữ liệu huấn luyện và trong chính các mô hình AI, nhằm mục đích tạo ra các đầu ra công bằng hơn, bình đẳng hơn và ít phân biệt đối xử hơn trên các nhóm dân cư đa dạng.
*   **Minh bạch, khả năng giải thích và đóng dấu watermark:** Tăng cường minh bạch trong cách hệ thống AI tạo ra đầu ra ("AI có thể giải thích" hoặc XAI) và khám phá các phương pháp đóng dấu watermark hoặc ký kỹ thuật số cho nội dung do AI tạo ra là những lĩnh vực trọng tâm để xây dựng niềm tin, cho phép xác minh và phân biệt đầu ra của AI với sản phẩm sáng tạo của con người.
*   **Hiệu quả năng lượng và tối ưu hóa:** Các nỗ lực đang được tiến hành để phát triển các kiến trúc mô hình AI hiệu quả năng lượng hơn, tối ưu hóa thuật toán huấn luyện và cải thiện cơ sở hạ tầng phần cứng và trung tâm dữ liệu nền tảng để giảm tác động môi trường đáng kể và chi phí vận hành của việc triển khai AI quy mô lớn.
*   **Phát triển và thích ứng lực lượng lao động:** Các tổ chức và chính phủ đang nhận ra nhu cầu cấp thiết phải đầu tư mạnh vào việc đào tạo lại và nâng cao kỹ năng cho lực lượng lao động. Điều này chuẩn bị cho nhân viên cộng tác hiệu quả *với* các công nghệ AI, tập trung vào phát triển các kỹ năng tận dụng những điểm mạnh độc đáo của con người (sự sáng tạo, đánh giá phản biện, sự đồng cảm) cùng với khả năng của AI, nuôi dưỡng một tương lai làm việc được tăng cường bởi AI.

**8. Kết luận**

Tính đến đầu năm 2025, AI tạo sinh đang ở trong trạng thái năng động của sự tiến bộ nhanh chóng và tích hợp ngày càng tăng trên một loạt các lĩnh vực đa dạng trên toàn cầu. Mặc dù nó mang đến tiềm năng to lớn để thúc đẩy đổi mới, đạt được những lợi ích đáng kể về năng suất, cho phép những trải nghiệm được cá nhân hóa cao và giải quyết các vấn đề phức tạp, nó đồng thời đi kèm với những thách thức đáng kể về kỹ thuật, đạo đức và xã hội. Việc vượt qua thành công những trở ngại này, bao gồm giải quyết thiên vị dữ liệu, quản lý "ảo giác" và độ tin cậy, giảm thiểu chi phí cao và tác động môi trường, đề cao các nguyên tắc đạo đức và quyền riêng tư, đảm bảo an ninh, và giải quyết các tác động xã hội như thay thế việc làm và thông tin sai lệch, là điều tối quan trọng. Sự cân nhắc kỹ lưỡng, nỗ lực hợp tác giữa các bên liên quan và các chiến lược giảm thiểu chủ động là điều cần thiết để đảm bảo việc phát triển và triển khai các công nghệ AI tạo sinh có trách nhiệm, công bằng, an toàn và cuối cùng là mang lại lợi ích cho tương lai.

---

## ThaiAgent

แน่นอนครับ นี่คือรายงานฉบับแปลเป็นภาษาไทย พร้อมการปรับปรุงเล็กน้อยตามคำขอ เพื่อให้เนื้อหาไหลลื่นและชัดเจนยิ่งขึ้น:

---

**รายงานฉบับสมบูรณ์: สถานะปัจจุบันของปัญญาประดิษฐ์เชิงสร้างสรรค์ (ณ ต้นปี 2568)**

**บทสรุปสำหรับผู้บริหาร**

ณ ต้นปี 2568 ปัญญาประดิษฐ์เชิงสร้างสรรค์ (Generative Artificial Intelligence - Generative AI) เป็นเทคโนโลยีที่มีการพัฒนาอย่างรวดเร็วและสร้างการเปลี่ยนแปลงในเชิงลึกอย่างมาก เทคโนโลยีนี้ก้าวพ้นจากระยะเริ่มต้นสู่การถูกนำไปใช้อย่างแพร่หลายในอุตสาหกรรมต่างๆ มากมาย โดยผนวกเข้ากับกระบวนการทางธุรกิจหลักและแอปพลิเคชันสำหรับผู้บริโภค ขับเคลื่อนด้วยแบบจำลองขั้นสูง เช่น แบบจำลองภาษาขนาดใหญ่ (LLMs) และความสามารถแบบหลากหลายรูปแบบ (multimodal capabilities) ที่เพิ่มขึ้นอย่างต่อเนื่อง ปัญญาประดิษฐ์เชิงสร้างสรรค์มอบศักยภาพที่สำคัญสำหรับนวัตกรรม การปรับแต่งเฉพาะบุคคล และการเพิ่มผลิตภาพ อย่างไรก็ตาม ความก้าวหน้าอย่างรวดเร็วนี้มาพร้อมกับความท้าทายที่สำคัญ รวมถึงอคติของข้อมูล "ภาวะหลอน" (hallucinations) ต้นทุนการประมวลผลที่สูง ข้อกังวลด้านจริยธรรมและความเป็นส่วนตัว และผลกระทบทางสังคมในวงกว้าง เช่น การแทนที่งานและความเสี่ยงด้านความปลอดภัย การจัดการกับข้อจำกัดเหล่านี้ผ่านกรอบจริยธรรมที่แข็งแกร่ง การกำกับดูแลที่มีประสิทธิภาพ และกลยุทธ์การบรรเทาที่มุ่งเน้น จึงเป็นสิ่งสำคัญอย่างยิ่งในการตระหนักถึงศักยภาพของเทคโนโลยีนี้อย่างมีความรับผิดชอบและเท่าเทียม

**1. บทนำสู่ปัญญาประดิษฐ์เชิงสร้างสรรค์**

ปัญญาประดิษฐ์เชิงสร้างสรรค์ คือ ปัญญาประดิษฐ์รูปแบบที่ซับซ้อน โดดเด่นด้วยความสามารถในการสร้างเนื้อหา **ใหม่** ซึ่งแตกต่างอย่างสิ้นเชิงกับปัญญาประดิษฐ์แบบดั้งเดิมที่เน้นการวิเคราะห์หรือจัดหมวดหมู่ข้อมูล **ที่มีอยู่** เป็นหลัก เนื้อหาใหม่นี้สามารถอยู่ในรูปแบบต่างๆ ได้แก่ ข้อความ รูปภาพ เสียง โค้ด และแม้กระทั่งโครงสร้าง 3 มิติ กลไกพื้นฐานเกี่ยวข้องกับการเรียนรู้รูปแบบ โครงสร้าง และการกระจายตัวที่ซับซ้อนจากชุดข้อมูลขนาดใหญ่ เพื่อสร้างผลลัพธ์ที่เป็นของใหม่ทั้งหมด แต่ยังคงสอดคล้องกับลักษณะของข้อมูลที่ใช้ฝึก การเปลี่ยนแปลงกระบวนทัศน์นี้ส่วนใหญ่ขับเคลื่อนด้วยความก้าวหน้าอย่างมีนัยสำคัญในสถาปัตยกรรมแบบจำลองพื้นฐาน โดยเฉพาะอย่างยิ่ง เครือข่ายปฏิปักษ์เชิงกำเนิด (Generative Adversarial Networks - GANs) ตัวเข้ารหัสอัตโนมัติแบบแปรผัน (Variational Autoencoders - VAEs) และที่โดดเด่นที่สุดคือ แบบจำลองภาษาขนาดใหญ่ (Large Language Models - LLMs)

**2. สถานะปัจจุบันของการนำมาใช้และการบูรณาการ**

ภาพรวมปัจจุบันของปัญญาประดิษฐ์เชิงสร้างสรรค์โดดเด่นด้วยการนำไปใช้ที่แพร่หลายและเร่งตัวขึ้นอย่างรวดเร็วทั้งในภาคธุรกิจและผู้บริโภค องค์กรต่างๆ กำลังเปลี่ยนผ่านจากระยะการสำรวจเบื้องต้น และหันมาให้ความสำคัญกับการบูรณาการปัญญาประดิษฐ์เชิงสร้างสรรค์เข้ากับกระบวนการทำงานหลักและแอปพลิเคชันระดับองค์กรอย่างมีความหมายมากขึ้น การเปลี่ยนแปลงเชิงกลยุทธ์นี้แสดงให้เห็นจากการลงทุนที่เพิ่มขึ้นอย่างมากในเทคโนโลยีปัญญาประดิษฐ์เชิงสร้างสรรค์ในทุกภาคส่วน แง่มุมสำคัญของการนำมาใช้นี้คือ การฝังฟังก์ชันปัญญาประดิษฐ์เชิงสร้างสรรค์โดยตรงเข้าในซอฟต์แวร์และบริการที่มีอยู่ ฟังก์ชันเหล่านี้มักปรากฏในรูปแบบของส่วนต่อประสานเชิงสนทนาที่ใช้งานง่าย ผู้ช่วยดิจิทัลที่ซับซ้อน แชตบอตที่ให้ความช่วยเหลือ และ "โคไพลอต" (copilots) ทรงพลังที่ออกแบบมาเพื่อเสริมศักยภาพมนุษย์ในงานต่างๆ ซึ่งช่วยเพิ่มประสิทธิภาพและปลดล็อกความเป็นไปได้ใหม่ๆ

**3. แนวโน้มและความสามารถหลักที่กำหนดภาพรวม**

มีแนวโน้มหลักหลายประการที่กำลังกำหนดภาพรวมปัญญาประดิษฐ์เชิงสร้างสรรค์ในปัจจุบัน:

*   **การเพิ่มขึ้นของ AI หลากหลายรูปแบบ (Multimodal AI):** การพัฒนาที่สำคัญคือ การเกิดขึ้นของแบบจำลอง AI หลากหลายรูปแบบขั้นสูง แบบจำลองเหล่านี้มีความสามารถในการประมวลผลและสร้างเนื้อหาข้ามรูปแบบที่แตกต่างกันได้อย่างราบรื่นพร้อมกัน เช่น การผนวกรวมข้อความ รูปภาพ เสียง วิดีโอ และแม้กระทั่งข้อมูล 3 มิติ ความสามารถในการทำความเข้าใจและการสร้างเนื้อหาข้ามรูปแบบนี้ช่วยขยายขอบเขตของแอปพลิเคชันที่เป็นไปได้สำหรับปัญญาประดิษฐ์เชิงสร้างสรรค์ให้ก้าวข้ามผลลัพธ์แบบรูปแบบเดียวไปอย่างมาก
*   **การปรับแต่งเฉพาะบุคคลขั้นสูง (Hyper-Personalization):** ปัญญาประดิษฐ์เชิงสร้างสรรค์กำลังทำให้เกิดการปรับแต่งเฉพาะบุคคลในระดับที่ไม่เคยมีมาก่อนในอุตสาหกรรมต่างๆ ภาคส่วนอย่างการตลาด อีคอมเมิร์ซ การดูแลสุขภาพ และเกม กำลังใช้ประโยชน์จากแบบจำลองเชิงสร้างสรรค์เพื่อปรับแต่งเนื้อหา คำแนะนำ และประสบการณ์ให้ตรงกับความต้องการ ประวัติ และความจำเป็นเฉพาะของผู้ใช้แต่ละราย สร้างการโต้ตอบที่มีส่วนร่วมและเกี่ยวข้องสูง
*   **แอปพลิเคชันแบบเรียลไทม์:** ประสิทธิภาพที่เพิ่มขึ้นและความล่าช้าที่ลดลงของแบบจำลองเชิงสร้างสรรค์กำลังนำไปสู่การใช้งานแอปพลิเคชันเชิงสร้างสรรค์แบบเรียลไทม์ที่แพร่หลายมากขึ้น แอปพลิเคชันเหล่านี้ช่วยอำนวยความสะดวกในงานแบบไดนามิก เช่น การแปลภาษาระหว่างการสนทนาได้ทันที การสร้างเนื้อหาแบบไดนามิกอย่างรวดเร็วสำหรับกิจกรรมสดหรือแคมเปญการตลาดส่วนบุคคล และการให้การโต้ตอบที่ตอบสนองสูงและรับรู้บริบทในสถานการณ์บริการลูกค้าและการสนับสนุน

**4. แนวโน้มในอนาคตและด้านที่กำลังเกิดขึ้น**

เมื่อมองไปในอนาคตอันใกล้ มีการคาดการณ์ว่าจะมีการพัฒนาที่โดดเด่นมากขึ้นดังนี้:

*   **เอไอเอเจนต์อัตโนมัติ (Autonomous AI Agents):** คาดการณ์ว่าจะมีการเกิดขึ้นและการใช้เอไอเอเจนต์อัตโนมัติที่ขับเคลื่อนด้วยแบบจำลองเชิงสร้างสรรค์อย่างแพร่หลายมากขึ้น เอเจนต์เหล่านี้ถูกออกแบบมาให้มีความสามารถในการทำความเข้าใจเป้าหมายที่ซับซ้อน แบ่งย่อยออกเป็นงาน ดำเนินการงานเหล่านั้น และตัดสินใจได้เองโดยอัตโนมัติหรือกึ่งอัตโนมัติ โดยต้องการการแทรกแซงจากมนุษย์น้อยที่สุดสำหรับบางฟังก์ชัน
*   **เอไอเฉพาะทาง/เอไอแนวตั้ง (Domain-Specific/Vertical AI):** มีแนวโน้มที่เห็นได้ชัดเจนในการเน้นแอปพลิเคชันปัญญาประดิษฐ์เชิงสร้างสรรค์เพื่อแก้ปัญหาเฉพาะที่ซับซ้อนภายในอุตสาหกรรมใดอุตสาหกรรมหนึ่งโดยเฉพาะ การเน้น "เอไอแนวตั้ง" นี้กำลังเร่งการพัฒนาแบบจำลองและแอปพลิเคชันเชิงสร้างสรรค์ที่เชี่ยวชาญ ซึ่งปรับให้เหมาะกับข้อกำหนด กฎระเบียบ และข้อมูลเฉพาะของภาคส่วนต่างๆ เช่น การดูแลสุขภาพ (เช่น การค้นพบยา การวินิจฉัยโรค) การเงิน (เช่น การตรวจจับการฉ้อโกง การวิเคราะห์ตลาด) หรือการผลิต (เช่น การเพิ่มประสิทธิภาพการออกแบบ การบำรุงรักษาเชิงคาดการณ์)
*   **ความก้าวหน้าด้านฮาร์ดแวร์และการประมวลผล:** ความก้าวหน้าอย่างต่อเนื่องในด้านฮาร์ดแวร์ โดยเฉพาะอย่างยิ่งในตัวเร่งความเร็วสำหรับเอไอโดยเฉพาะ (เช่น GPUs, TPUs) และโครงสร้างพื้นฐานที่ได้รับการปรับปรุงให้เหมาะสม ควบคู่ไปกับพลังการประมวลผลที่เพิ่มขึ้น กำลังมีบทบาทสำคัญ ความก้าวหน้าเหล่านี้มีส่วนอย่างมากต่อความคืบหน้าอย่างรวดเร็ว ความสามารถที่เพิ่มขึ้น ประสิทธิภาพที่ดีขึ้น และความสามารถในการขยายขนาดที่มากขึ้นของแบบจำลองปัญญาประดิษฐ์เชิงสร้างสรรค์ ทำให้แอปพลิเคชันที่เคยทำไม่ได้ในอดีตกลายเป็นไปได้

**5. ข้อจำกัดและความท้าทายที่สำคัญ**

แม้จะมีความก้าวหน้าอย่างรวดเร็วและศักยภาพในการเปลี่ยนแปลงที่สูง ปัญญาประดิษฐ์เชิงสร้างสรรค์ในปัจจุบันยังคงเผชิญกับข้อจำกัดและความท้าทายที่สำคัญหลายประการ ซึ่งเป็นอุปสรรคต่อการนำไปใช้อย่างไร้ข้อผิดพลาดและการใช้งานอย่างแพร่หลายที่น่าเชื่อถือ:

*   **การพึ่งพาข้อมูลและอคติ:** ความท้าทายหลักมาจากการที่เทคโนโลยีนี้พึ่งพาชุดข้อมูลขนาดใหญ่ที่ใช้ในการฝึก อคติที่มีอยู่ (ทางสังคม ประวัติศาสตร์ การเป็นตัวแทน) ภายในชุดข้อมูลเหล่านี้ สามารถถูกสืบทอดและขยายใหญ่ขึ้นโดยแบบจำลอง ซึ่งอาจนำไปสู่ผลลัพธ์ที่มีลักษณะเลือกปฏิบัติ ไม่เป็นธรรม หรือไม่เกี่ยวข้อง โดยเฉพาะอย่างยิ่งส่งผลกระทบต่อกลุ่มหรือภูมิภาคที่ขาดการเป็นตัวแทน การระบุและบรรเทาอคติเหล่านี้เป็นเรื่องซับซ้อน แต่มีความสำคัญอย่างยิ่ง
*   **ภาวะหลอนและความน่าเชื่อถือ:** ปรากฏการณ์ "ภาวะหลอน" ซึ่งแบบจำลองสร้างข้อมูลที่ฟังดูน่าเชื่อถือ แต่ไม่ถูกต้อง สร้างขึ้นมาเอง หรือไร้สาระโดยสิ้นเชิง ยังคงเป็นปัญหาสำคัญ สิ่งนี้ส่งผลกระทบโดยตรงต่อความน่าเชื่อถือ ความถูกต้องตามข้อเท็จจริง และความไว้วางใจได้ของเนื้อหาที่สร้างโดยเอไอ ทำให้จำเป็นต้องมีการตรวจสอบยืนยัน การตรวจสอบข้อเท็จจริง และการกำกับดูแลโดยมนุษย์อย่างระมัดระวัง โดยเฉพาะอย่างยิ่งในแอปพลิเคชันที่ละเอียดอ่อน
*   **การขาดแหล่งความจริงและการให้เหตุผล:** ต่างจากมนุษย์ แบบจำลองปัญญาประดิษฐ์เชิงสร้างสรรค์โดยเนื้อแท้ขาด "แหล่งความจริง" หรือแบบจำลองโลกที่แท้จริงที่เป็นอิสระ พวกมันมักมีปัญหาในการทำความเข้าใจบริบทที่ละเอียดอ่อน ความสัมพันธ์เชิงสาเหตุ และการให้เหตุผลที่ซับซ้อนและหลายขั้นตอนอย่างแท้จริง โดยมักพึ่งพารูปแบบทางสถิติที่เรียนรู้จากข้อมูล มากกว่าการอนุมานเชิงตรรกะที่แท้จริงหรือข้อเท็จจริงที่ได้รับการตรวจสอบแล้ว
*   **ทรัพยากรการประมวลผลและความยั่งยืน:** การฝึกและการรันแบบจำลองเชิงสร้างสรรค์ขนาดใหญ่ต้องการทรัพยากรการประมวลผลและพลังงานจำนวนมาก ซึ่งนำไปสู่ต้นทุนโครงสร้างพื้นฐานที่สำคัญ และก่อให้เกิดข้อกังวลอย่างมากเกี่ยวกับความยั่งยืนด้านสิ่งแวดล้อม เนื่องจากมีรอยเท้าคาร์บอน (carbon footprints) สูงที่เกี่ยวข้องกับการใช้พลังงาน
*   **ปัญหาด้านจริยธรรมและความเป็นส่วนตัว:** ข้อกังวลด้านจริยธรรมและความเป็นส่วนตัวที่สำคัญยิ่งยวด ได้แก่ ศักยภาพในการรั่วไหลของข้อมูลโดยไม่ตั้งใจหรือโดยเจตนาร้าย (ของข้อมูลที่ใช้ฝึกหรือข้อมูลนำเข้า) การนำข้อมูลส่วนบุคคลหรือข้อมูลที่ละเอียดอ่อนไปใช้ในทางที่ผิด การขาดความโปร่งใสหรือ "ความสามารถในการอธิบาย" (explainability) ในการที่แบบจำลองได้มาซึ่งผลลัพธ์ และศักยภาพในการสร้างเนื้อหาที่เป็นอันตราย หลอกลวง หรือทำให้เข้าใจผิด เช่น ดีปเฟคที่ซับซ้อน โฆษณาชวนเชื่อที่พุ่งเป้า และการรณรงค์เผยแพร่ข้อมูลเท็จอย่างกว้างขวาง
*   **ความยากลำบากในการบูรณาการ:** การบูรณาการความสามารถของปัญญาประดิษฐ์เชิงสร้างสรรค์เข้ากับระบบเก่าที่มีอยู่หลากหลาย กระบวนการทำงานระดับองค์กรที่ซับซ้อน และโครงสร้างข้อมูลขององค์กร อาจเป็นเรื่องท้าทายทางเทคนิคและมีค่าใช้จ่ายสูง ซึ่งเป็นอุปสรรคอย่างยิ่งสำหรับวิสาหกิจขนาดกลางและขนาดย่อม (SMEs) ที่มีผู้เชี่ยวชาญด้านเทคนิคและทรัพยากรจำกัด
*   **ข้อกังวลด้านทรัพย์สินทางปัญญา:** การจัดการสิทธิ์ในทรัพย์สินทางปัญญา (IPR) เป็นเรื่องซับซ้อน ก่อให้เกิดคำถามทางกฎหมายและจริยธรรมที่สำคัญเกี่ยวกับความเป็นเจ้าของเนื้อหาที่สร้างโดยเอไอ และศักยภาพของแบบจำลองเชิงสร้างสรรค์ที่จะละเมิดเนื้อหาที่มีลิขสิทธิ์ที่ใช้ในการฝึกข้อมูลหรือนำมารวมในผลลัพธ์โดยไม่ตั้งใจ ซึ่งบางครั้งก็เกิดขึ้นโดยไม่ได้รับความยินยอมอย่างชัดเจน
*   **อุปสรรคทางภาษา:** แม้จะมีความคืบหน้า แต่ความเหลื่อมล้ำด้านประสิทธิภาพและข้อจำกัดมักยังคงมีอยู่ระหว่างภาษา สำเนียง และความละเอียดอ่อนทางวัฒนธรรมที่แตกต่างกัน แบบจำลองมักจะทำงานได้ดีที่สุดในภาษาที่มีการเป็นตัวแทนจำนวนมากในข้อมูลที่ใช้ฝึก (ส่วนใหญ่เป็นภาษาอังกฤษ) ซึ่งนำไปสู่ผลลัพธ์ที่น่าเชื่อถือน้อยลงหรือไม่เหมาะสมทางวัฒนธรรมในบริบททางภาษาอื่นๆ

**6. ข้อกังวลทางสังคมในวงกว้าง**

นอกเหนือจากความท้าทายทางเทคนิคและจริยธรรม ความสามารถที่เพิ่มขึ้นและการนำไปใช้ที่แพร่หลายของปัญญาประดิษฐ์เชิงสร้างสรรค์ยังก่อให้เกิดข้อกังวลทางสังคมในวงกว้างอีกด้วย:

*   **การกัดเซาะความคิดสร้างสรรค์และทักษะของมนุษย์:** การพึ่งพาเครื่องมือ AI มากเกินไปสำหรับงานต่างๆ เช่น การเขียน การออกแบบ และการเขียนโค้ด อาจนำไปสู่การกัดเซาะความคิดสร้างสรรค์พื้นฐาน ทักษะการคิดเชิงวิพากษ์ การพัฒนาทักษะ และความเป็นต้นฉบับในอุตสาหกรรมที่ต้องใช้ความคิดสร้างสรรค์และความรู้ หากไม่ได้ใช้เป็นเครื่องมือเสริม
*   **การแทนที่งานและการปรับเปลี่ยนรูปแบบการทำงาน:** มีความเสี่ยงอย่างมีนัยสำคัญของการแทนที่งานในภาคส่วนที่พึ่งพางานที่ปัญญาประดิษฐ์เชิงสร้างสรรค์สามารถทำให้เป็นอัตโนมัติหรือเสริมได้อย่างมาก โดยเฉพาะอย่างยิ่งส่งผลกระทบต่อบทบาทที่เกี่ยวข้องกับการสร้างเนื้อหา การวิเคราะห์ข้อมูล บริการลูกค้า และหน้าที่ทางการบริหารบางอย่าง สิ่งนี้จำเป็นต้องมีการให้ความสำคัญกับการปรับเปลี่ยนรูปแบบการทำงานและการปรับตัวของกำลังแรงงาน
*   **ความเสี่ยงด้านความปลอดภัย:** ปัญญาประดิษฐ์เชิงสร้างสรรค์สามารถเพิ่มความเสี่ยงด้านความปลอดภัยที่มีอยู่ โดยให้ความสามารถที่เพิ่มขึ้นแก่ผู้ไม่หวังดีในการดำเนินการหลอกลวงที่ซับซ้อน การโจมตีแบบฟิชชิ่ง การฉ้อโกง การขโมยข้อมูลประจำตัว การโจมตีทางไซเบอร์แบบอัตโนมัติ และการรณรงค์เผยแพร่ข้อมูลเท็จขนาดใหญ่ที่น่าเชื่อถือสูง ซึ่งออกแบบมาเพื่อชักจูงความคิดเห็นสาธารณะหรือสร้างความแตกแยก

**7. การตอบสนองและกลยุทธ์การบรรเทา**

ด้วยตระหนักถึงความท้าทายและข้อกังวลเหล่านี้ จึงมีการให้ความสำคัญกับการดำเนินมาตรการเชิงรุกจากนักวิจัย ผู้พัฒนา ผู้กำหนดนโยบาย และองค์กรต่างๆ มากขึ้น:

*   **กรอบจริยธรรม AI และการกำกับดูแล:** มีความพยายามอย่างมากในการพัฒนากรอบจริยธรรม AI ที่แข็งแกร่ง แนวปฏิบัติอุตสาหกรรม แนวปฏิบัติที่ดีที่สุด และการจัดตั้งโครงสร้างการกำกับดูแลและกฎระเบียบระดับชาติและนานาชาติที่มีประสิทธิภาพ เพื่อนำทางการพัฒนา การนำไปใช้ และการใช้งานปัญญาประดิษฐ์เชิงสร้างสรรค์อย่างมีความรับผิดชอบ โดยให้ความสำคัญกับความเป็นอยู่ที่ดีของมนุษย์และประโยชน์ต่อสังคม
*   **การบรรเทาอคติและความเป็นธรรม:** มีการวิจัยและพัฒนาอย่างจริงจังที่เน้นเทคนิคในการระบุ วัดผล และบรรเทาอคติที่มีอยู่ในข้อมูลที่ใช้ฝึกและภายในแบบจำลอง AI เอง โดยมีเป้าหมายเพื่อให้ได้ผลลัพธ์ที่เป็นธรรม เที่ยงธรรม และมีลักษณะเลือกปฏิบัติน้อยลงในกลุ่มประชากรที่หลากหลาย
*   **ความโปร่งใส ความสามารถในการอธิบาย และการใส่ลายน้ำ:** การเพิ่มความโปร่งใสในวิธีการที่ระบบ AI สร้างผลลัพธ์ ("เอไอที่อธิบายได้" หรือ XAI) และการสำรวจวิธีการในการใส่ลายน้ำหรือลงนามดิจิทัลในเนื้อหาที่สร้างโดยเอไอ เป็นประเด็นสำคัญที่มุ่งเน้นเพื่อสร้างความไว้วางใจ อนุญาตให้มีการตรวจสอบยืนยัน และแยกแยะผลงานของเอไอออกจากผลงานของมนุษย์
*   **ประสิทธิภาพด้านพลังงานและการเพิ่มประสิทธิภาพ:** มีความพยายามในการพัฒนาสถาปัตยกรรมแบบจำลอง AI ที่ใช้พลังงานอย่างมีประสิทธิภาพมากขึ้น ปรับปรุงอัลกอริทึมการฝึกให้เหมาะสมที่สุด และปรับปรุงฮาร์ดแวร์และโครงสร้างพื้นฐานศูนย์ข้อมูลพื้นฐาน เพื่อลดผลกระทบต่อสิ่งแวดล้อมและต้นทุนการดำเนินงานที่สำคัญของการนำ AI ไปใช้ในขนาดใหญ่
*   **การพัฒนาและการปรับตัวของกำลังแรงงาน:** องค์กรและรัฐบาลกำลังตระหนักถึงความจำเป็นอย่างยิ่งในการลงทุนอย่างมากในการฝึกอบรมและเพิ่มทักษะใหม่ให้กับบุคลากร เพื่อเตรียมความพร้อมให้พนักงานสามารถทำงานร่วม **กับ** เทคโนโลยี AI ได้อย่างมีประสิทธิภาพ โดยเน้นการพัฒนาทักษะที่ใช้ประโยชน์จากจุดแข็งเฉพาะตัวของมนุษย์ (ความคิดสร้างสรรค์ วิจารณญาณเชิงวิพากษ์ ความเห็นอกเห็นใจ) ควบคู่ไปกับความสามารถของ AI เพื่อส่งเสริมอนาคตของการทำงานที่เสริมด้วยเอไอ

**8. บทสรุป**

ณ ต้นปี 2568 ปัญญาประดิษฐ์เชิงสร้างสรรค์อยู่ในสถานะที่เปลี่ยนแปลงอย่างรวดเร็วและมีการบูรณาการเพิ่มขึ้นในหลากหลายภาคส่วนทั่วโลก แม้ว่าจะนำเสนอศักยภาพอันมหาศาลในการขับเคลื่อนนวัตกรรม บรรลุประสิทธิภาพการทำงานที่เพิ่มขึ้นอย่างมาก สร้างประสบการณ์ที่ปรับแต่งเฉพาะบุคคลอย่างสูง และแก้ปัญหาที่ซับซ้อน แต่ก็มาพร้อมกับความท้าทายทางเทคนิค จริยธรรม และสังคมที่สำคัญไปพร้อมกัน การฝ่าฟันอุปสรรคเหล่านี้ให้สำเร็จ รวมถึงการจัดการกับอคติของข้อมูล การจัดการ "ภาวะหลอน" และความน่าเชื่อถือ การบรรเทาต้นทุนที่สูงและผลกระทบต่อสิ่งแวดล้อม การยึดมั่นในหลักจริยธรรมและความเป็นส่วนตัว การรับรองความปลอดภัย และการจัดการกับผลกระทบทางสังคม เช่น การแทนที่งานและข้อมูลเท็จ เป็นสิ่งสำคัญอย่างยิ่ง การพิจารณาอย่างรอบคอบ ความพยายามร่วมกันจากผู้มีส่วนได้ส่วนเสียทั้งหมด และกลยุทธ์การบรรเทาเชิงรุก เป็นสิ่งจำเป็นอย่างยิ่งเพื่อให้แน่ใจว่าการพัฒนาและการนำเทคโนโลยีปัญญาประดิษฐ์เชิงสร้างสรรค์ไปใช้จะเป็นไปอย่างรับผิดชอบ เท่าเทียม ปลอดภัย และท้ายที่สุดก็เป็นประโยชน์ต่ออนาคต

---

## Loop Agents

Some workflow involves repetition or iterative refinement, such as like revising code. ADK also support such workflows.



In [33]:
# Declare constants
APP_NAME = "loop_app" # New App Name
USER_ID = "loop_user"
SESSION_ID = "loop_session" # New Base Session ID
STATE_INITIAL_TOPIC = "Kopi Luwak"

# Define state variables
STATE_CURRENT_DOC = "current_document"
STATE_CRITICISM = "criticism"

# Define the exact phrase the Critic should use to signal completion
COMPLETION_PHRASE = "No major issues found."

#### Tools

In [34]:
# Define exit loop tool
def exit_loop(tool_context: ToolContext):
    """Call this function ONLY when the critique indicates no further changes are needed, signaling the iterative process should end."""
    print(f"[Tool Call] exit_loop triggered by {tool_context.agent_name}")
    tool_context.actions.escalate = True
    # Return empty dict as tools should typically return JSON-serializable output
    return {}

#### Initial Writer Agent

In [35]:
# STEP 1: Initial Writer Agent (Runs ONCE at the beginning)
initial_writer_agent = LlmAgent(
    name="InitialWriterAgent",
    model=GEMINI_MODEL_NAME,
    include_contents='none',
    instruction=f"""You are a Creative Writing Assistant tasked with starting a story.
    Write the *first draft* of a short story (aim for 2-4 sentences).
    Base the content *only* on the topic provided below. Try to introduce a specific element (like a character, a setting detail, or a starting action) to make it engaging.
    Topic: {{initial_topic}}

    Output *only* the story/document text. Do not add introductions or explanations.
""",
    description="Writes the initial document draft based on the topic, aiming for some initial substance.",
    output_key=STATE_CURRENT_DOC
)

#### Critic Agent

In [36]:
# STEP 2a: Critic Agent (Inside the Refinement Loop)
critic_agent_in_loop = LlmAgent(
    name="CriticAgent",
    model=GEMINI_MODEL_NAME,
    include_contents='none',
    # MODIFIED Instruction: More nuanced completion criteria, look for clear improvement paths.
    instruction=f"""You are a Constructive Critic AI reviewing a short document draft (typically 2-6 sentences). Your goal is balanced feedback.

    **Document to Review:**
    ```
    {{current_document}}
    ```

    **Task:**
    Review the document for clarity, engagement, and basic coherence according to the initial topic (if known).

    IF you identify 1-2 *clear and actionable* ways the document could be improved to better capture the topic or enhance reader engagement (e.g., "Needs a stronger opening sentence", "Clarify the character's goal"):
    Provide these specific suggestions concisely. Output *only* the critique text.

    ELSE IF the document is coherent, addresses the topic adequately for its length (should be 1 page long), and has no glaring errors or obvious omissions:
    Respond *exactly* with the phrase "{COMPLETION_PHRASE}" and nothing else. It doesn't need to be perfect, just functionally complete for this stage. Avoid suggesting purely subjective stylistic preferences if the core is sound.

    Do not add explanations. Output only the critique OR the exact completion phrase.
""",
    description="Reviews the current draft, providing critique if clear improvements are needed, otherwise signals completion.",
    output_key=STATE_CRITICISM
)

### Refine Agent

In [37]:
# STEP 2b: Refiner/Exiter Agent (Inside the Refinement Loop)
refiner_agent_in_loop = LlmAgent(
    name="RefinerAgent",
    model=GEMINI_MODEL_NAME,
    # Relies solely on state via placeholders
    include_contents='none',
    instruction=f"""You are a Creative Writing Assistant refining a document based on feedback OR exiting the process.
    **Current Document:**
    ```
    {{current_document}}
    ```
    **Critique/Suggestions:**
    {{criticism}}

    **Task:**
    Analyze the 'Critique/Suggestions'.
    IF the critique is *exactly* "{COMPLETION_PHRASE}":
    You MUST call the 'exit_loop' function. Do not output any text.
    ELSE (the critique contains actionable feedback):
    Carefully apply the suggestions to improve the 'Current Document'. Output *only* the refined document text.

    Do not add explanations. Either output the refined document OR call the exit_loop function.
""",
    description="Refines the document based on critique, or calls exit_loop if critique indicates completion.",
    tools=[exit_loop], # Provide the exit_loop tool
    output_key=STATE_CURRENT_DOC # Overwrites state['current_document'] with the refined version
)

#### Refinement Loop Agent

In [38]:
# STEP 2c: Refinement Loop Agent
refinement_loop = LoopAgent(
    name="RefinementLoop",
    # Agent order is crucial: Critique first, then Refine/Exit
    sub_agents=[
        critic_agent_in_loop,
        refiner_agent_in_loop,
    ],
    max_iterations=5 # Limit loops
)

#### Overall Sequential Agent

In [39]:
# STEP 3: Overall Sequential Pipeline
# For ADK tools compatibility, the root agent must be named `root_agent`
root_agent = SequentialAgent(
    name="IterativeWritingPipeline",
    sub_agents=[
        initial_writer_agent, # Run first to create initial doc
        refinement_loop       # Then run the critique/refine loop
    ],
    description="Writes an initial document and then iteratively refines it with critique using an exit tool."
)

In [47]:
# Initialize Session
session_service = InMemorySessionService()

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state={"initial_topic": "Kopi Luwak"}
)

# Initialize Runner
runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,
)

In [49]:
await run_agent(runner, USER_ID, SESSION_ID, "write a comprehensive essay")

## InitialWriterAgent

Old Man Budi scanned the forest floor, his eyes sharp despite the morning mist, searching for the tell-tale droppings among the ferns and leaves. He knew the value hidden within the civet's waste – the unique beans destined for the world's most expensive cup. Each dark nugget he found felt like discovering a small, aromatic jewel in the damp earth. This was the strange beginning of Kopi Luwak.

## CriticAgent

No major issues found.

[Tool Call] exit_loop triggered by RefinerAgent


{'InitialWriterAgent': "Old Man Budi scanned the forest floor, his eyes sharp despite the morning mist, searching for the tell-tale droppings among the ferns and leaves. He knew the value hidden within the civet's waste – the unique beans destined for the world's most expensive cup. Each dark nugget he found felt like discovering a small, aromatic jewel in the damp earth. This was the strange beginning of Kopi Luwak.",
 'CriticAgent': 'No major issues found.'}

# Conclusions

You have explored how to create an agent using Gemini and Agent Development Kit.